## Encontrando dados e usando data wrangling para series temporais

In [167]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

### Readaptando uma coleção de dados de Séries temporais a partir de uma conjunto de dados.

Vamos imaginar que voce trabalha em uma grande organização sem fins lucrativos e tem monitorado uma variedade de fatores que podem ser uteis que podem ser uteis para a analise de séries temporais:
* A reação de um destinário de e-mail em relação aos e-mails que recebeu no decorrer do tempo: ele abriu os e-mails ou não?
* Historico de afiliação: houve periodos em que um membro deixou de ser afiliado?
* Historio de transações? quando alguem compra e como predizer isso?

> Voce pode analisar os dados de diversas formas que serão ensinadas mais adiante

#### Exemplo prático da sua organização sem fins lucrativos

`Ano em que cada membro se afilou e o status atual de membro`

In [168]:
year_joined = pd.read_csv('dataset/year_joined.csv')
year_joined.head()

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018


`Número de e-mails enviados em uma determinada semana e abertos pelo menbro`

In [169]:
email_opened = pd.read_csv('dataset/emails.csv')
email_opened.head()

,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00


`tempo em que os membros doaram a sua organização`

In [170]:
donation = pd.read_csv('dataset/donations.csv')
donation.head()

,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0
3,50.0,2016-07-06 12:24:55,0.0
4,50.0,2016-05-11 18:13:04,1.0


> É possivel responder perguntas relacionada ao tempo com base nos dados que temos. Contudo, se não colocarmos em um formato de série temporal adequado, não é possivel obter comportamento mais granulares além de não conseguir responder perguntas

Note que os dados estão em tres niveis temporais:
1. Status anual do membro
2. Registro semanal de e-mail abertos
3. Timestamps instantaneos de doações

In [171]:
year_joined.groupby(by=['user']).count().groupby(by=['userStats']).count()

,yearJoined
userStats,
1,1000


> estamos verificando se há repetição na entrada de um usuario, no caso de todos os mil membro tem apenas um status, oq siginifica que o ano se afiliaram provavelmente é o `yearjoined`

Na tabela `email_open` temos a coluna week quanto seu conteudo que os dados tem um timestamp ou período de tempo semanal. Isso deve ser agregado aolongo da semana,assim temos considerar esses timestamps com períodos semanais em vez de timestamps que ocorrem com uma semana de diferença

Verificando se há semanas nulas no dataset

In [172]:
email_opened[email_opened['emailsOpened'] < 1]

,emailsOpened,user,week


> Por mais que não tenha, não faz sentido um usuario abrir todos os seu emails sem deixar passar pelo menos uma vez.
Para tal problema abordado vamos analisar o historio de um usuario

In [173]:
email_opened[email_opened['user'] == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


Verificando matematicamente sé ha alguma semana ausente

In [174]:
email_opened['week'] = pd.to_datetime(email_opened['week'])

In [175]:
email_opened[email_opened['emailsOpened'] == 998]

,emailsOpened,user,week


In [176]:
# email_opened['week'][email_opened['emailsOpened'] == 998]
subset = email_opened[email_opened['user'] == 998]

(subset['week'].max() - subset['week'].min()).days/7

25.0

In [177]:
subset.shape

(24, 3)

> Temos 24 linhas, mas deveriamos ter 26 - isso mostra que há algumas semanas faltando.
>
> Por que 26 linhas?
> Exemplo:
> $$ dias(7, 14, 21, 28) = \frac{28 - 7}{7} = 3$$
> Contudo falta o contablizar o dia 28 no caso são 4 dias

Vamos preencher as semanas nulas nos dados

In [178]:
completo_index = pd.MultiIndex.from_product((
    set(email_opened['week']), set(email_opened['user'])
))

In [179]:
completo_index[0:10]

MultiIndex([('2015-06-15',  1.0),
            ('2015-06-15',  3.0),
            ('2015-06-15',  5.0),
            ('2015-06-15',  6.0),
            ('2015-06-15',  9.0),
            ('2015-06-15', 10.0),
            ('2015-06-15', 14.0),
            ('2015-06-15', 16.0),
            ('2015-06-15', 20.0),
            ('2015-06-15', 21.0)],
           )

> Nesse código, você está criando um índice múltiplo (completo_index) a partir do produto cartesiano de dois conjuntos de valores:
>
> * set(email_opened['week']): extrai os valores únicos da coluna week do DataFrame email_opened e os converte em um conjunto.
> * set(email_opened['user']): extrai os valores únicos da coluna user do DataFrame email_opened e os converte em um conjunto.


In [180]:
todos_emails = email_opened.set_index(['week', 'user'])\
                            .reindex(completo_index, fill_value=0)\
                            .reset_index()

# a coluna user de valores numericos sumiu
todos_emails.head(2)

,level_0,level_1,emailsOpened
0,2015-06-15,1.0,0.0
1,2015-06-15,3.0,0.0


In [181]:
todos_emails.columns = ['week', 'user','emailsOpened']
todos_emails.head(2)

,week,user,emailsOpened
0,2015-06-15,1.0,0.0
1,2015-06-15,3.0,0.0


Olhando o usuario 998

In [182]:
todos_emails[todos_emails['user'] == 998].head()

,week,user,emailsOpened
538,2015-06-15,998.0,0.0
1077,2016-02-29,998.0,0.0
1616,2015-09-28,998.0,0.0
2155,2016-01-18,998.0,0.0
2694,2017-05-22,998.0,0.0


In [183]:
todos_emails[todos_emails['user'] == 998].shape

(173, 3)

In [184]:
todos_emails.dtypes

week            datetime64[ns]
user                   float64
emailsOpened           float64
dtype: object

In [185]:
todos_emails['emailsOpened'].value_counts()

emailsOpened
0.0    67759
3.0    15350
1.0     5070
2.0     5068
Name: count, dtype: int64

Temos uma grande numero de zeros no inicio, antes a afiliação. Além disso, depois da ultima atualização dessa forma vamos definir um ponto de corte para remoção desses 0


In [186]:
# Exemplo
email_opened.groupby(['user'])['week'].agg(['min', 'max']).reset_index().head()

,user,min,max
0,1.0,2015-06-29,2018-05-28
1,3.0,2018-03-05,2018-04-23
2,5.0,2017-06-05,2018-05-28
3,6.0,2016-12-05,2018-05-28
4,9.0,2016-07-18,2018-05-28


In [187]:
for _, row in email_opened.groupby(['user'])['week'].agg(['min', 'max']).reset_index().iterrows():
    user_ = row['user']
    min_ = row['min']
    max_ = row['max']

    todos_emails.drop(
        index = todos_emails[(todos_emails['user'] == user_) & (todos_emails['week'] < min_)].index,
        inplace = True
    )

    todos_emails.drop(
        index = todos_emails[(todos_emails['user'] == user_) & (todos_emails['week'] > max_)].index,
        inplace = True
    )

In [188]:
email_clean = todos_emails.copy()

In [189]:
email_clean[email_clean['user'] == 998].shape

(26, 3)

### Contruindo uma serie temporal encontrada

Relacionando os dados de email e doações entre si. 

In [190]:
donation.timestamp = pd.to_datetime(donation.timestamp)
donation.set_index('timestamp', inplace=True)

In [191]:
donation.head()

,amount,user
timestamp,,
2017-11-12 11:13:44,25.0,0.0
2015-08-25 19:01:45,50.0,0.0
2015-03-26 12:03:47,25.0,0.0
2016-07-06 12:24:55,50.0,0.0
2016-05-11 18:13:04,50.0,1.0


In [192]:
agg = donation.groupby(['user']).apply(
    lambda x: x.amount.resample('W-MON').sum().dropna()
).reset_index()

C:\Users\mateu\AppData\Local\Temp\ipykernel_11852\4183698632.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg = donation.groupby(['user']).apply(


> Para cada user, soma o amount por semanas que terminam na segunda-feira (W-MON) e remove semanas sem valor (NaN).

In [193]:
def exemplo():
    donation = pd.DataFrame({
    'user': [1, 1, 1, 2, 2],
    'date': [
        '2024-01-01', '2024-01-02', '2024-01-08',
        '2024-01-01', '2024-01-09'
    ],
    'amount': [10, 5, 7, 3, 4]
    })

    donation['date'] = pd.to_datetime(donation['date'])
    donation.set_index('date', inplace=True)

    return donation.groupby(['user']).apply(
        lambda x: x.amount.resample('W-MON').sum().dropna()
    )
exemplo()

C:\Users\mateu\AppData\Local\Temp\ipykernel_11852\1087864188.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return donation.groupby(['user']).apply(


user  date      
1     2024-01-01    10
      2024-01-08    12
2     2024-01-01     3
      2024-01-08     0
      2024-01-15     4
Name: amount, dtype: int64

In [213]:
user_donation.dtypes

user      float64
amount    float64
dtype: object

In [221]:
merge_data = pd.DataFrame(
    columns=['user', 'week', 'amount', 'emailsOpened']
)
pd.set_option('future.no_silent_downcasting', True)
for user_, member_email in todos_emails.groupby(['user']):

    # Verificando se o usuário doou
    user_donation = agg[agg.user == user_]
    if user_ == 998.0:
        print(user_donation)
        break
    # Definindo a coluna 'timestamp' com indice
    user_donation.set_index('timestamp', inplace=True)

    member_email.set_index('week', inplace=True)

    # 
    member_email = todos_emails[todos_emails['user'] == user_]

    member_email.sort_values('week').set_index('week', inplace=True)
    
    data = member_email.sort_values('week').set_index('week')\
        .merge(
            user_donation, how='left', left_index=True, right_index=True
        )


    data = data.fillna(0)

    data['user']  = data['user_x']

    merge_data = pd.concat(
        [merge_data, data.reset_index()[['user', 'week', 'amount', 'emailsOpened']]]
    )

C:\Users\mateu\AppData\Local\Temp\ipykernel_11852\2740211370.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merge_data = pd.concat(


In [225]:
subset = merge_data[merge_data['user']==998].copy(deep=True)
subset['target'] = subset['amount'].shift(1)
subset.head(10)

,user,week,amount,emailsOpened,target
0,998.0,2017-12-04,0.0,1.0,NaN
1,998.0,2017-12-11,0.0,3.0,0.0
2,998.0,2017-12-18,0.0,3.0,0.0
3,998.0,2017-12-25,0.0,0.0,0.0
4,998.0,2018-01-01,0.0,3.0,0.0
5,998.0,2018-01-08,50.0,3.0,0.0
6,998.0,2018-01-15,0.0,2.0,50.0
7,998.0,2018-01-22,0.0,3.0,0.0
8,998.0,2018-01-29,0.0,2.0,0.0
9,998.0,2018-02-05,0.0,3.0,0.0
